#### Sequential Machine Learning App


In [2818]:
#Imports

from resample.jackknife import resample as b_resample
from numpy import random
import ipywidgets as widgets
from ipywidgets import RadioButtons,FloatRangeSlider,Button,IntSlider
from IPython.display import display,Markdown
import pandas as pd
import sys
import statistics
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

#FUELS-imports
from lolopy.learners import RandomForestRegressor
from operator import add
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap

In [2819]:
#Components

tab = widgets.Tab()    

out = widgets.Output(layout={'border': '1px solid black'})
out_plotting= widgets.Output(layout={'border': '1px solid black'})


out_algo=widgets.Output(layout={'border': '1px solid black'})
out_algo_alt=widgets.Output(layout={'border': '1px solid black'})
out_plot_hist=widgets.Output(layout={'border': '1px solid black'})

out_perform_experiment=widgets.Output(layout={'border': '1px solid black'})

out_res=widgets.Output(layout={'border': '1px solid black'})


up = widgets.FileUpload(accept="", multiple=False)

delim = widgets.RadioButtons(
    options=[';', ',', ' '],
    description='Separator: ',
    disabled=False)

delim_dec = widgets.RadioButtons(
    options=['.', ','],
    description='Decimal Delim: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"',"%"],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

##ML-###pre processing

feature_selector=widgets.SelectMultiple(
    options=[],
    #tooltip("Select the Input Features")
    #rows=10,
    description='Features',
    disabled=False,
layout=widgets.Layout(width='50%'))
    
target_selection=widgets.SelectMultiple(
    options=[],
    #value='',
    placeholder='select the Target Variable',
    description='Target-Variable',
    disabled=False,
layout=widgets.Layout(width='50%'))

selector_plot_variable=widgets.SelectMultiple(
    options=[],
    #tooltip("Select the Input Features")
    #rows=10,
    description='Features',
    disabled=False,
layout=widgets.Layout(width='50%'))
    





graph_type = widgets.Dropdown(
    options=['Choose the Graph Type','Scatter', 'Pairwise Relation', 'Correlation Heatmap','TSNE'],
    value='Choose the Graph Type',
    description='Chart Type:',
    disabled=False,
    layout=widgets.Layout(width='50%'))

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)
toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])

select_x=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select X-Axis',
    description='X-Axis:',
    disabled=False,
layout=widgets.Layout(width='50%'))
select_y=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select Y-Axis',
    description='Y-Axis:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_hue=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select the hue',
    description='Hue:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_size=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select the Size',
    description='Size:',
    disabled=False,
layout=widgets.Layout(width='50%'))


select_strategy=widgets.Dropdown(
    options=['MEI','MU','MLI','Customize Strategy'],
    value='MEI',
    placeholder='select the strategy',
    description='Strategy:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_model=widgets.Dropdown(
    options=['Random Forrest (RF)','Decision Trees (DT)','Bagged Tree Ensembles (TE)','Gaussian Process Regression (GPR)'],
    value='Decision Trees (DT)',
    placeholder='select the Model',
    description='Model:',
    disabled=False,
layout=widgets.Layout(width='50%'))




button_confirm_strategy=widgets.Button(
    description='Confirm Strategy ',
    disabled=False,
    button_style='success',
    tooltip='Confirm selected Strategy',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))




#Options
box_features=widgets.VBox([])




#container_feature_options=widgets.VBox([widgets.HBox([box_features,box_features_slider]),box_confirm])
#feature_options= widgets.Accordion(children=[container_feature_options])

#feature_options.set_title(0,"Specify your Target")

In [2820]:
#Buttons Anfang
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='success',
    tooltip='Click to Upload',
    icon='check',
layout=widgets.Layout(width='50%'))

button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search',
layout=widgets.Layout(width='50%'))



#Plotting
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='warning',
    tooltip='Click to Plot',
    icon='pencil',
layout=widgets.Layout(width='100%'))




###Algos
button_confirm_feature=widgets.Button(
    description='Confirm Features ',
    disabled=False,
    button_style='success',
    tooltip='Confirm Feature Selection of Training Features',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))

button_confirm_target=widgets.Button(
    description='Confirm Target',
    disabled=False,
    button_style='success',
    tooltip='Confirm the selected Target Variable',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))

button_confirm_plot_var=widgets.Button(
    description='Confirm Target',
    disabled=False,
    button_style='success',
    tooltip='Confirm the selected Target Variable',
    icon='check',
    layout=widgets.Layout(width='50%',height ='inherit'))










button_plot_comparision=widgets.Button(
    description='Compare',
    disabled=False,
    button_style='warning',
    tooltip='Simplify the Columns',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='50%')
)


button_confirm_options=Button(
    description='Confirm Options ',
    disabled=False,
    button_style='success',
    tooltip='Confirm options',
    icon='check',
    layout=widgets.Layout(width='50%',height ='inherit'))
                           
###Algos ende
##ML End

button_custom_strategey=widgets.Button(
    description='Customize',
    disabled=False,
    button_style='warning',
    tooltip='Simplify the Columns',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='50%')
    
    
    
)
button_perform_experiment=widgets.Button(
    description='Perform Experiment',
    disabled=False,
    button_style='warning',
    tooltip='Perform Experiment',
    icon='fa-calculator',
    layout=widgets.Layout(width='100%'))
    
#Buttons Ende

In [2821]:
#FileUpload Tab
accordion = widgets.Accordion(children=[
    up, 
    widgets.VBox([delim, delim_dec, eraser]), 
    rows])

accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')


accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload ]),
    out
])

In [2822]:
#Sequential Learning Tab 

quantile_tar_slider= widgets.FloatSlider(
    value=90,
    min=0,
    max=100,
    step=1,
    description='Quantile Target:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)


quantile_sample_slider= widgets.FloatSlider(
    value=50,
    min=0,
    max=100,
    step=5,
    description='Quantile Sample:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)
initial_sample_size_text=widgets.BoundedIntText(
    value=4,
    min=0,
    max=10,
    step=1,
    description='Initial Sample Size:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

#slider_quantile_container=widgets.VBox([widgets.HBox(children=[quantile_tar_slider,quantile_sample_slider,initial_sample_size_text])])

box_features_slider=widgets.VBox([])


DataPre_sl=widgets.VBox([
        widgets.HBox([feature_selector, button_confirm_feature ]),
        widgets.HBox([target_selection, button_confirm_target]),
#      slider_quantile_container,
#feature_options,
#button_split_data,
    box_features,
    quantile_tar_slider,
    quantile_sample_slider,
    initial_sample_size_text,

out_data_pre_pr])

DataPre=widgets.VBox([
        widgets.HBox([feature_selector, button_confirm_feature ]),
        widgets.HBox([target_selection, button_confirm_target]),
       out_data_pre_pr,
])

iterations=widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=5,
    description='Iterations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='50%')
)
show_results_button= widgets.Button(
    description='Show Results',
    disabled=False,
    button_style='warning',
    tooltip='Show Results',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='100%')
)

custom_container=widgets.VBox([widgets.HBox([])])
results_container=widgets.VBox([widgets.HBox([])])

sl_settings= widgets.VBox([
    widgets.HBox([select_strategy, button_confirm_strategy ]),
    custom_container,
    widgets.HBox([select_model,iterations]),
    widgets.HBox([button_perform_experiment]),
    out_perform_experiment
   ,results_container,
    out_plot_hist
    
    
])



sl_accordion=widgets.Accordion(children=[DataPre_sl,sl_settings])
sl_accordion.set_title(0,"Data Pre Processing")
sl_accordion.set_title(1,"Model Building")


In [2823]:
#Plotting Tab



container_plot_options= widgets.VBox([])
button_container=widgets.HBox([button_plot])

plotting=widgets.VBox(children=[widgets.VBox( [
        widgets.HBox([graph_type]),
        container_plot_options,
        button_container,
        out_plotting
        
        
]


)])

children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    plotting,
    #plot_accordion,
    sl_accordion
    
   ]

In [2824]:
#Tabs
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Describer")
tab.set_title(2, "Plotter")

tab.set_title(3,"Sequential Learning")

In [2825]:
#EventHandling

def content_parser():
    if up.value == {}:
        """with out:
            out.clear_output
            display(Markdown('No CSV loaded'))
            #print('No CSV loaded')    """
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')

                if eraser.value != {}: 
                    for val in eraser.value:
                        if val == "tab":
                            content_str = content_str.replace("\t","")
                        elif val =="%":
                            content_str = content_str.replace("\t","")
                        else:
                            content_str = content_str.replace(val,"")
                if content_str != "":
                    str_io = StringIO(content_str) 
                    return str_io
def df_converter():
    content = content_parser()
    if content is not None:
            df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value,decimal=delim_dec.value)
            df=df.apply(pd.to_numeric,errors="coerce")
            
            
            
                
            return df
    else:
        return None
def preview():
    
    df = df_converter()
    with out:
        out.clear_output()
        display(Markdown('Now this is how your DF looks like:'))
        
        if df is not None:
            print(df.head(10))
        else:
            display(Markdown('Configuration is wrong/missing...'))
            
def upload():
    
    df = df_converter()
    with out:
        out.clear_output()
        display(Markdown('Your uploaded DF looks like:'))
       
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
            feature_selector.options= df.columns
            select_x.options=df.columns
            select_y.options=df.columns
            select_size.options=df.columns
            select_hue.options=df.columns
            selector_plot_variable.options=df.columns
            
        else:
            display(Markdown('Configuration is wrong/missing...'))
            #print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    display(Markdown('Configuration is wrong/missing...'))
    
    

    
#plot methods   



def plot():
    graph = graph_type.value
    if graph=="Scatter":
        plot_scatter()
    elif graph=="Correlation Heatmap":
            plot_heat()
    elif graph=="Pairwise Relation":
            plot_pairwise()
    
    
        
        

def plot_pairwise():
    
    df =confirm_var()
    
    with out_plotting:
        out_plotting.clear_output()
        sns.pairplot(df)
        plt.show()

def plot_heat():
    
    df = confirm_var()
    
    with out_plotting:
        out_plotting.clear_output()
        corr = df.corr()

        plt.figure(figsize=(9,7))
        sns.heatmap(corr, annot=True, cmap='Blues')
        b, t = plt.ylim()
        plt.ylim(b+0.5, t-0.5)
        plt.title("Feature Correlation Heatmap")
        plt.show()
            
def plot_scatter():
    data=df_converter()
    with out_plotting:
        out_plotting.clear_output()
        fig, ax = plt.subplots(figsize=(10,7))
        #not generic
        sns.scatterplot(y=select_y.value, x=select_x.value, hue=select_hue.value, size=select_size.value, data=data, ax=ax, sizes=(50, 300))
        ax.set_title(select_y.value+ "vs"+ select_x.value)
        ax.legend(loc="upper left", bbox_to_anchor=(1,1))
        plt.show()
    

In [2826]:
#ml eventhandling
#pre pocessing



def confirm_options():
    items=box_features_slider.children
    df = df_converter()
    Y = df.loc[:,df.columns.isin(target_selection.value)]
    
    for slider in items:
            unt_grenz= slider.value[0]/100
            ob_grenz= slider.value[1]/100
            Y = Y[(Y >= Y.quantile(unt_grenz) ) & (Y <= Y.quantile(ob_grenz))]
            Y= Y.dropna()
    
    return Y
    
    
#returns index of features
def confirm_features():
    df = df_converter()
    train = feature_selector.value
    
    target_selection.options=df.columns[~df.columns.isin(feature_selector.value)]
    
    train = df.columns[df.columns.isin(feature_selector.value)]
    
    print("train in confirm")
    print(train)
    print("type train in confirm")
    print(type(train))
    return train
    
def confirm_var():
    df= df_converter()
    
    
    #df.columns[df.columns.isin(feature_selector.value)]
    #var = df.columns[df.columns.isin(selector_plot_variable.value)]
    
    selection = list(selector_plot_variable.value)
    
    var = df[selection]
    
    
    
    
    
   
    
    
    return var


    
#returns index of target    
def confirm_target():
    df = df_converter()
    #target_Var=target_selection.value
    #feature_selector.options= get_features()
    target = df.columns[df.columns.isin(target_selection.value)]
    
    
    if(len(box_features.children) == 0):
        create_dynamically_checkboxes()
   
    
    return target 
    
    

def create_dynamically_checkboxes():
    target= target_selection.value
    items = [RadioButtons(
    options=['maximize', 'minimize'],
    value='maximize', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description=feature,
    disabled=False
) for feature in target]
    box_features.children=items
        


In [2827]:
class sequential_learning:
    
    xlabel="Sequential Learning Iteration"
    dataframe = df_converter()
    init_sample_size=4
    number_of_executions=0
    features_df=df_converter()
    target_df=df_converter()
    targ_q=quantile_tar_slider.value
    samp_q=quantile_sample_slider.value
    
    tries_list=[]
    
    y_pred_dtr_mean=None
    y_pred_dtr_std=None
    y_pred_dtr=None
    SampIdx=None
    PredIdx=None
        
    
    
    def __init__(self,dataframe):  #constructor
        self.dataframe= dataframe
        
    
    def apply_feature_selection_to_df(self,dataframe):
        self.features_df = self.dataframe[confirm_features()]
        print("self.features_df in apply")
        print(self.features_df)
        
    
    def apply_target_selection_to_df(self,dataframe):
        self.target_df= self.dataframe[confirm_target()]
        
        
        
        
    def normalize_data(self):
        
        
        
        dataframe_norm=(self.dataframe-self.dataframe.mean())/self.dataframe.std()
        target_df_norm=(self.target_df-self.target_df.mean())/self.target_df.std()
        
        features_df_norm=(self.features_df-self.features_df.mean())/self.features_df.std()
        
        
        self.features_df=features_df_norm
        self.target_df=target_df_norm
        self.dataframe=dataframe_norm
        return self.features_df, self.target_df, self.dataframe
        
    
    def set_init_sample_size(self):
        self.init_sample_size=initial_sample_size_text.value
    
        
    def set_number_of_ex(self):
        self.number_of_executions= iterations.value
        
        
    def init_sampling(self):
         
        self.samp_q= quantile_sample_slider.value/100
        
        samp_q_t=self.target_df.quantile(self.samp_q)
        
        
        
       
        
        
        Index_label=np.where(self.dataframe[confirm_target()] < samp_q_t )
        
        
       
        Index_label=Index_label[0]
        
        
        init_sample_set = np.ones((0,self.init_sample_size))
        
       
    
        for i in range(self.number_of_executions):
                init_sample_set=np.vstack([init_sample_set, random.choice(Index_label,self.init_sample_size)])
                
        return init_sample_set
    
    def decide_max_or_min(self):
        max_or_min_optimizations_list=[]
        
        for decide in range(len(self.target_df.columns)):
            
                            
            max_or_min_optimizations_list.append(box_features.children[decide].value)
        
        return max_or_min_optimizations_list
            
    def start_sequential_learning(self):
        with out_perform_experiment:
                display(Markdown('The experiment get executed... this could take some a while...'))
       
        print("self.features_df b4 init in learning")
        print(self.features_df)
            
        init_sample_set=self.init_sampling()
        print("self.features_df b4 seq learning")
        print(self.features_df)
        
        for i in range(self.number_of_executions):
            
            self.SampIdx=init_sample_set[i].astype(int)
            self.PredIdx=self.dataframe
            self.PredIdx = self.PredIdx.drop(self.PredIdx.index[self.SampIdx]).index
               
            ##Train Initial Model
            if select_model.value == 'Random Forrest (RF)':
                    print("RF")
            elif select_model.value == 'Decision Trees (DT)':
                    self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_DT_wJK()
                    
                    
            elif select_model.value == 'Bagged Tree Ensembles (TE)':
                    print("TE")
            elif select_model.value == 'Gaussian Process Regression (GPR)':
                    print("GPR")
                
                
            if(i%5==0):
                with out_perform_experiment:
                    self.plot_hist()
                    


            
            self.tries_list.append(self.init_sample_size)
                
            
            
            
            decide_optimization=self.decide_max_or_min()
           
            for k in decide_optimization:
                index_of_target = decide_optimization.index(k)
                
                if (k == "minimize"):
                   
                    
                    self.target_df.iloc[:,index_of_target]*(-1)
                    
            self.targ_q = quantile_tar_slider.value/100
            targ_q_t= self.target_df.quantile(self.targ_q)
            
            
                    
            
            while self.target_df.sum(axis =1).max() <= targ_q_t.values.sum():

                        

                            #Update_Idx_with_Strategy 

                            if confirm_strategy()=='MEI':
                                      self.updateIndexMEI()

                            elif confirm_strategy()=='MU':
                                    self.updateIndexMU()

                            elif confirm_strategy()=='MLI':
                                   self.updateIndexMLI()
                            elif confirm_strategy()=='Customize Strategy':
                                    print("implement")

                            ##Train Model

                            if select_model.value == 'Random Forrest (RF)':
                                print("RF")
                            elif select_model.value == 'Decision Trees (DT)':
                                self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_DT_wJK()


                            elif select_model.value == 'Bagged Tree Ensembles (TE)':
                                print("TE")
                            elif select_model.value == 'Gaussian Process Regression (GPR)':
                                print("GPR")


                            print("tries")
                            print(self.tries_list[i])
                            self.tries_list[i]=self.tries_list[i]+1

    
    
            
        with out_perform_experiment:
            display(Markdown('Sequential Learning performed ✅'))
        print("self.features_df after seq learning")
        print(self.features_df)
            

        results_container.children=[show_results_button]
        
                
            
        
          
              
                
            

 


 

    
    def updateIndexMEI(self):
            print(index_max)
            print(self.PredIdx)
            index_max = np.argmax(self.y_pred_dtr_mean)

            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            new_SampIdx=np.append(self.SampIdx,index_max)
            self.SampIdx=new_SampIdx
            
            
    def updateIndexMU(self):
            index_max = np.argmax(self.y_pred_dtr_std)
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            new_SampIdx=np.append(self.SampIdx,index_max)
            self.SampIdx=new_SampIdx
            
            
    def updateIndexMLI(self):
            mli=self.y_pred_dtr_mean+self.y_pred_dtr_std
            index_max = np.argmax(mli)
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            new_SampIdx=np.append(self.SampIdx,index_max)
            self.SampIdx=new_SampIdx
        
           
        
           
            
            
                
    def plot_TSNE_input_space(self):
        
        
        print("features_df")
        print(self.features_df)
        print("X")
        print(X)
        print("Type x")
        print(type(X))
        
        y=self.target_df
        print("y")
        print(y)
        tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
        tsne_results = tsne.fit_transform(X)
        
 

        # Plot Results in reduced FS

        with out_res:
            plt.figure(figsize=(16,10))
            color = [y] # y sind hier die labels
            sc=plt.scatter(x=tsne_results[:,0], y=tsne_results[:,1], s=200, c=color)
            palette=sns.color_palette("hls")
            plt.colorbar(sc)

                
    def fit_DT_wJK(self):
        
        
       
        td= []
        for x in b_resample(self.features_df.iloc[self.SampIdx]):
            td.append(x)
        tl=[]
        for x in b_resample(self.target_df.iloc[self.SampIdx]): 
            tl.append(x)
            
        
        td=np.array(td)
        tl=np.array(tl)
        self.y_pred_dtr=[]
        for i in range(len(td)):
            dtr = DecisionTreeRegressor()
            dtr.fit(td[i], tl[i])
            
            self.y_pred_dtr.append(dtr.predict(self.features_df.iloc[self.PredIdx]))
        
        self.y_pred_dtr=np.array(self.y_pred_dtr)
        self.y_pred_dtr=self.y_pred_dtr.T
        row_means = self.y_pred_dtr.mean(axis=1)
        row_std = self.y_pred_dtr.std(axis=1)
        
        
        
        return self.y_pred_dtr,row_means, row_std
        
    def fit_TE_wJK():
        print("implement")
        
    def plot_hist(self):
        
            plt.figure(figsize=(10, 6))
            #plt.xlabel('')
            #plt.ylabel("Tries to find optimal Input Data")
            plt.title("Performance of %s with strategy %s"%(select_model.value,select_strategy.value))
            plt.hist(self.tries_list,range(iterations.value))
            plt.show()
        
        
    def main(self):
        self.apply_feature_selection_to_df(self.dataframe)
        self.apply_target_selection_to_df(self.dataframe)
        self.normalize_data()
        self.set_init_sample_size()
        self.set_number_of_ex()
        init_sample_set=self.init_sampling()
        self.start_sequential_learning()
        self.plot_TSNE_input_space()
        
    
    
        
       
                
        
        


In [2828]:
text = widgets.Text(value='Hello World',
            placeholder='',
            description='Customize Strategy:',
            disabled=False)

customize = Button(description='Customize',
                    disabled=False,
                    button_style='warning',
                    tooltip='Click to Confirm Customization',
                    icon='fa-wrench ',
                    layout=widgets.Layout(width='50%'))

def confirm_strategy():
    strategy= select_strategy.value
    if strategy != {}:
        if strategy =='Customize Strategy':
            custom_container.children=[text,customize]
        else:
            custom_container.children=[]
            return select_strategy.value
                
                
                
def add_custom_to_strategy():
    select_strategy.options.append(text.value)

            
        
    

In [2829]:
df=df_converter()
s= sequential_learning(df)


In [2830]:
######EventListener######### 

###Uploads Anfang####


def preview_clicked(b):
    try:
        preview()
    except pd.errors.ParserError:
        with out:
            out.clear_output
            display(Markdown("Sth. wennt wrong! Please check your csv and the upload settings"))
    
def upload_clicked(b):
    try:
        upload()
    except pd.errors.ParserError:
        with out:
            out.clear_output
            display(Markdown("Sth. wennt wrong! Please check your csv and the upload settings"))
def desc_clicked(b):
    desc()

    
###Uploads Ende####


###Plot Anfang####
def plotter_clicked(b):
    plot()

def pairwise_clicked(b):
    plot_pairwise()
def heat_clicked(b):
    plot_heat()
def scatter_clicked(b):
    plot_scatter()
    
###Plot Ende####

    
###Selection SL Anfang###

def confirm_features_clicked(b):
    confirm_features()
    
def confirm_target_clicked(b):
    confirm_target()
    
def on_graph_type_change(change):
        
    if graph_type.value =="Scatter":
        container_plot_options.children= [widgets.HBox([select_x,select_y]),
        widgets.HBox([select_hue,select_size])]
    elif graph_type.value =="Pairwise Relation":
        
        container_plot_options.children= [widgets.HBox([selector_plot_variable,button_confirm_plot_var])]
                                          
    elif graph_type.value =='Correlation Heatmap':
        container_plot_options.children= [widgets.HBox([selector_plot_variable,button_confirm_plot_var])]
        
    else: container_plot_options.children=[]


def confirm_var_clicked(b):
    confirm_var()

def confirm_options_clicked(b):
    confirm_options()
    
def customize_clicked(b):
    add_custom_to_strategy()
def confirm_strategy_clicked(b):
    confirm_strategy()
    
def perform_experiment_clicked(b):
    s= sequential_learning(df_converter())
    s.main()
    
def show_results_button_clicked(b):
    results_container.children=[show_results_button,out_res]
    
    
def on_value_change(change):
    quantile_sample_slider.max=quantile_tar_slider.value



In [2831]:
#Eventss
#plot

graph_type.observe(on_graph_type_change,names='value')


button_preview.on_click(preview_clicked)


button_confirm_plot_var.on_click(confirm_var_clicked)


quantile_tar_slider.observe(on_value_change,names='value')


show_results_button.on_click(show_results_button_clicked)

button_confirm_feature.on_click(confirm_features_clicked)
button_confirm_target.on_click(confirm_target_clicked)

#sl


button_confirm_options.on_click(confirm_options_clicked)

button_confirm_strategy.on_click(confirm_strategy_clicked)

customize.on_click(customize_clicked)


button_perform_experiment.on_click(perform_experiment_clicked)



#upload etc

button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')

button_plot.on_click(plotter_clicked)

In [2832]:
tab

train in confirm
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate '],
      dtype='object')
type train in confirm
<class 'pandas.core.indexes.base.Index'>
train in confirm
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate '],
      dtype='object')
type train in confirm
<class 'pandas.core.indexes.base.Index'>
self.features_df in apply
      cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0      540.0                 0.0      0.0  162.0               2.5   
1      540.0                 0.0      0.0  162.0               2.5   
2      332.5               142.5      0.0  228.0               0.0   
3      332.5               142.5      0.0  228.0               0.0   
4      198.6               132.4      0.0  192.0               0.0   
...      ...                 ...      ...    ...               ...   
1025   276.4               1

NameError: name 'X' is not defined